In [1]:
import pandas as pd
import seaborn as sns
from IPython.display import display, HTML
import matplotlib.pyplot as plt
import scipy.stats as stats
import numpy as np
import nltk as nltk
import pyarrow
import bz2
import json
import datetime
from iteration_utilities import deepflatten
import nltk as nltk
import scipy.stats as stats
from nltk.sentiment.vader import SentimentIntensityAnalyzer
nltk.download('vader_lexicon')
pd.options.mode.chained_assignment = None
from textblob import TextBlob
from helpers import add_time, week, weekday_number, weekday, month, add_dict, to_dict

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\Benjamin\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


In [2]:
data_path = 'C:/Users/Benjamin/Desktop/Quotebank/'
files = ['quotes-2015.json.bz2','quotes-2016.json.bz2','quotes-2017.json.bz2',
                  'quotes-2018.json.bz2','quotes-2019.json.bz2','quotes-2020.json.bz2']

In [9]:
#defining sentiment analyzer
sent_analyzer = SentimentIntensityAnalyzer()

# Initializing dictionaries for dates, weekdays and months that will store values for all the chunks
date_dictionary = {}

# Dictionaries for weekdays and months are initialized with keys to keep them in the correct order for later plotting
weekday_dictionary = {'Monday': 0, 'Tuesday': 0, 'Wednesday': 0, 'Thursday': 0,
                      'Friday': 0, 'Saturday': 0, 'Sunday': 0}

month_dictionary = {'January': 0, 'February': 0,'March': 0,
                    'April': 0, 'May': 0, 'June': 0, 
                    'July': 0, 'August': 0, 'September': 0,
                    'October': 0, 'November': 0, 'December': 0}

# Iterating through all the cleaned data sets in chunks
for x in range(len(files)): 
  print('Working on %d. dataset'%(x+1))
  df_reader = pd.read_json(data_path + files[x], lines=True, compression='bz2', chunksize=50000)
  for chunk in df_reader:
    chunk = add_time(chunk)
    
    # Inserting values in the dictionaries
    date_dictionary = to_dict(date_dictionary, chunk['date_short'])
    weekday_dictionary = to_dict(weekday_dictionary, chunk['weekday'])
    month_dictionary = to_dict(month_dictionary, chunk['Month'])
    
    chunk['compound'] = chunk['quotation'].apply(lambda x: sent_analyzer.polarity_scores(x)['compound'])
    chunk['polarity'] = chunk['quotation'].apply(lambda x: TextBlob(x).sentiment[0])
    chunk['subjectivity'] = chunk['quotation'].apply(lambda x: TextBlob(x).sentiment[1])
    
    chunk = chunk[['date_short','compound','polarity','subjectivity','weekday', 'Month']]
    print(chunk.head())
    break
  break

Working on 1. dataset
   date_short  compound                                     polarity  \
0  2015-08-31    0.8020    (0.43333333333333335, 0.4499999999999999)   
1  2015-12-08    0.0000                                   (0.6, 1.0)   
2  2015-09-10   -0.8720  (-0.17500000000000002, 0.30000000000000004)   
3  2015-07-23    0.0000                   (0.16, 0.5399999999999999)   
4  2015-10-04   -0.1779                                   (0.0, 0.0)   

    weekday      Month  
0    Monday     August  
1   Tuesday   December  
2  Thursday  September  
3  Thursday       July  
4    Sunday    October  
